In [1]:
import numpy as np
import pandas as pd
import networkx as nx

# Spaces

In [2]:
tetrahedron = {
    0 : [0, 1, 2, 3],
    1 : [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)],
    2 : [(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)]
}

In [3]:
full_tetrahedron = {
    0 : [0, 1, 2, 3],
    1 : [(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3)],
    2 : [(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)],
    3 : [(0, 1, 2, 3)]
}

In [4]:
torus = {
    0 : [0, 1, 2, 3, 4, 5, 6, 7, 8],
    1 : [(0, 1), (1, 2), (2, 0), 
         (3, 5), (5, 7), (7, 3),
         (4, 6), (6, 8), (8, 4), 
         (0, 3), (3, 4), (4, 0),
         (1, 5), (5, 6), (6, 1), 
         (2, 7), (7, 8), (8, 2), 
         (1, 3), (2, 5), (5, 4),
         (0, 7), (7, 6), (6, 0),
         (3, 8), (8, 1), (4, 2)], 
    2 : [(0, 1, 3), (1, 3, 5), (1, 2, 5), 
         (2, 5, 7), (0, 2, 7), (0, 3, 7), 
         (3, 4, 5), (4, 5, 6), (5, 6, 7), 
         (6, 7, 8), (3, 7, 8), (3, 4, 8), 
         (0, 4, 6), (0, 1, 6), (1, 6, 8), 
         (1, 2, 8), (2, 4, 8), (0, 2, 4)]
}

In [5]:
torus_double = {}
#  дублируем тор
for i in range(3):
    torus_double.update({i : np.concatenate([np.array(torus[i]), np.array(torus[i])+9])})

# объеиняем по треугольнику
union_by = [6, 7, 8]
for i in range(3):
    m = torus_double[i]
    for j in union_by:
        m[m == j+9] = j
    torus_double[i] = m

torus_double[0] = np.unique(torus_double[0], axis=0)
for i in range(1, 3):
    torus_double[i] = np.unique(np.sort(torus_double[i], axis=1), axis=0)

# Убираем лишний треугольник
m = torus_double[2]
m = m == union_by
m = m.all(axis=1)
m = torus_double[2][np.logical_not(m)]
torus_double[2] = m

In [6]:
np.eye(3) == np.array([0, 1, 0])

array([[False, False,  True],
       [ True,  True,  True],
       [ True, False, False]])

In [7]:
mobius_strip = {
    0 : [0, 1, 2, 3, 4, 5],
    1 : [(0, 1), (1, 2), (0, 2), 
         (2, 3), (1, 3), (3, 4), 
         (2, 4), (4, 5), (3, 5), 
         (1, 4), (1, 5), (0, 5)],
    2 : [(0, 1, 2), (1, 2, 3), 
         (2, 3, 4), (3, 4, 5), 
         (1, 4, 5), (0, 1, 5)]
}

In [8]:
spaces = {
    'Empty Tetrahedron' : tetrahedron, 
    'Full Tetrahedron' : full_tetrahedron,
    'Torus' : torus, 
    'Double torus' : torus_double,
    'Möbius Strip' : mobius_strip
}

# Characteristic

In [9]:
def space_dimension(space):
    """
    Returns dimension of simplex space.
    
    Parameters:
    -----------
    space : dict
        Simplex space. Keys are dimensions, values are list of simplicies
    
    Returns:
    --------
    dimension : int
        Number of connected components.
    """
    dimension = np.max([dim for dim in space])
    return dimension

In [10]:
def number_connected_components(space):
    """
    Returns number of connected components.
    
    Parameters:
    -----------
    space : dict
        Simplex space. Keys are dimensions, values are list of simplicies
    
    Returns:
    --------
    num : int
        Number of connected components.
    """
    graph = nx.Graph()
    graph.add_nodes_from(space[0])
    graph.add_edges_from(space[1])
    num = nx.algorithms.components.number_connected_components(graph)
    return num

In [11]:
def euler_characteristic(space):
    """
    Returns Euler characterisitic of space.
    
    Parameters:
    -----------
    space : dict
        Simplex space. Keys are dimensions, values are list of simplicies
    
    Returns:
    --------
    char : int
        Euler characterisitic.
    """
    char = 0
    for dim in space:
        char += (-1)**dim * len(space[dim])
    return char

In [12]:
def get_chain_str(space, group='Z'):
    s = '0 <- '
    for dim in space:
        s += group + '^' + str(len(space[dim])) + ' <- '
    s += '0'
    return s

# Results

In [13]:
pd.DataFrame({
    'Space name' : [name for name in spaces], 
    'Dimension' : [space_dimension(spaces[name]) for name in spaces], 
    'Connected components' : [number_connected_components(spaces[name]) for name in spaces], 
    'Euler characterisitic' : [euler_characteristic(spaces[name]) for name in spaces], 
    'Chain' : [get_chain_str(spaces[name]) for name in spaces]
})

,Space name,Dimension,Connected components,Euler characterisitic,Chain
0,Empty Tetrahedron,2,1,2,0 <- Z^4 <- Z^6 <- Z^4 <- 0
1,Full Tetrahedron,3,1,1,0 <- Z^4 <- Z^6 <- Z^4 <- Z^1 <- 0
2,Torus,2,1,0,0 <- Z^9 <- Z^27 <- Z^18 <- 0
3,Double torus,2,1,-2,0 <- Z^15 <- Z^51 <- Z^34 <- 0
4,Möbius Strip,2,1,0,0 <- Z^6 <- Z^12 <- Z^6 <- 0
